In [46]:
import pandas as pd
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from sklearn.preprocessing import RobustScaler, LabelEncoder
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
import keras.backend as K

entorno = 'local'  # Elegir "VM" o "local" para correr en entorno local
cantidad_productos_cliente_mayor_ventas = 50
nombre_experimento = 'LSTM_producto_cliente'
nombre_archivo_log = 'log_predicciones.csv'
ventana_input = 3
lags = 3
ventana_output = 2

# Configurar entorno
if entorno == 'VM':
    carpeta_datasets = os.path.expanduser('~/buckets/b1/datasets')
    carpeta_exp_base = os.path.expanduser('~/buckets/b1/exp')
elif entorno == 'local':
    carpeta_datasets = 'C:\\Users\\alope\\Desktop\\Trámites\\Maestria Data Science - Universidad Austral\\Laboratorio de implementación 3\\Datos'
    carpeta_exp_base = 'C:\\Users\\alope\\Desktop\\Trámites\\Maestria Data Science - Universidad Austral\\Laboratorio de implementación 3\\Resultados'
else:
    raise Exception("Entorno especificado incorrectamente")

carpeta_exp = os.path.join(carpeta_exp_base, nombre_experimento)
if not os.path.exists(carpeta_exp):
    os.makedirs(carpeta_exp)
    
dataset_completo = pd.read_csv(os.path.join(carpeta_datasets, 'df_producto_cliente_completo.csv'))


dataset_completo.head()

,Timestamp,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,...,sku_size,descripcion,mes,quarter,fin_quarter,edad_producto,ventas_cat1,ventas_cat2,ventas_cat3,ventas_familia_producto
0,2017-01-01,10234,20524,0.0,2.0,0.05300,0.05300,HC,VAJILLA,Cristalino,...,500.0,Abrillantador,1,1,0,0,14.31686,4.96628,3.03194,0.25684
1,2017-02-01,10234,20524,NaN,0.0,0.00000,0.00000,HC,VAJILLA,Cristalino,...,500.0,Abrillantador,2,1,0,1,2.14290,0.10339,0.00000,0.00000
2,2017-03-01,10234,20524,0.0,1.0,0.01514,0.01514,HC,VAJILLA,Cristalino,...,500.0,Abrillantador,3,1,1,2,8.59237,2.23835,1.52777,0.04699
3,2017-04-01,10234,20524,NaN,0.0,0.00000,0.00000,HC,VAJILLA,Cristalino,...,500.0,Abrillantador,4,2,0,3,9.18260,4.47157,2.35257,0.00000
4,2017-05-01,10234,20524,NaN,0.0,0.00000,0.00000,HC,VAJILLA,Cristalino,...,500.0,Abrillantador,5,2,0,4,7.79714,0.50130,0.09348,0.00000


In [47]:
#Chequear si existe log de productos con su prediccion
if os.path.exists(os.path.join(carpeta_exp, nombre_archivo_log)):
    # Si existe archivo, leerlo
    productos_con_prediccion = pd.read_csv(os.path.join(carpeta_exp,nombre_archivo_log))
else:
    #Si el archivo no existe
    productos_con_prediccion = pd.DataFrame(columns = ['product_id' , 'customer_id', 'prediccion'])


# Crear una columna combinada para poder filtrar las combinaciones product/cliente ya predichas
productos_con_prediccion['product_customer_id'] = productos_con_prediccion['product_id'].astype(str) + '_' + productos_con_prediccion['customer_id'].astype(str)
dataset_completo['product_customer_id'] = dataset_completo['product_id'].astype(str) + '_' + dataset_completo['customer_id'].astype(str)

# Filtrar por la dupla customer_id y product_id
df = dataset_completo[~dataset_completo['product_customer_id'].isin(productos_con_prediccion['product_customer_id'].values)]

# Eliminar la columna combinada si ya no es necesaria
df = df.drop(columns=['product_customer_id'])
dataset_completo = dataset_completo.drop(columns=['product_customer_id'])
productos_con_prediccion = productos_con_prediccion.drop(columns=['product_customer_id'])

df

,Timestamp,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,...,sku_size,descripcion,mes,quarter,fin_quarter,edad_producto,ventas_cat1,ventas_cat2,ventas_cat3,ventas_familia_producto
0,2017-01-01,10234,20524,0.0,2.0,0.05300,0.05300,HC,VAJILLA,Cristalino,...,500.0,Abrillantador,1,1,0,0,14.31686,4.96628,3.03194,0.25684
1,2017-02-01,10234,20524,NaN,0.0,0.00000,0.00000,HC,VAJILLA,Cristalino,...,500.0,Abrillantador,2,1,0,1,2.14290,0.10339,0.00000,0.00000
2,2017-03-01,10234,20524,0.0,1.0,0.01514,0.01514,HC,VAJILLA,Cristalino,...,500.0,Abrillantador,3,1,1,2,8.59237,2.23835,1.52777,0.04699
3,2017-04-01,10234,20524,NaN,0.0,0.00000,0.00000,HC,VAJILLA,Cristalino,...,500.0,Abrillantador,4,2,0,3,9.18260,4.47157,2.35257,0.00000
4,2017-05-01,10234,20524,NaN,0.0,0.00000,0.00000,HC,VAJILLA,Cristalino,...,500.0,Abrillantador,5,2,0,4,7.79714,0.50130,0.09348,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6910867,2019-12-01,10083,20127,0.0,1.0,0.24645,0.24645,HC,ROPA LAVADO,Liquido,...,3000.0,Regular,12,4,1,3,35.48098,20.96524,6.93741,2.10633
6910868,2019-12-01,10074,20127,0.0,1.0,20.70148,20.70148,HC,ROPA LAVADO,Liquido,...,3000.0,Regular,12,4,1,3,113.11048,64.95754,49.02482,22.42660
6910869,2019-12-01,10172,20127,0.0,1.0,2.58769,2.58769,HC,ROPA LAVADO,Liquido,...,3000.0,Regular,12,4,1,3,14.09831,11.78360,11.78360,4.31281
6910870,2019-12-01,10175,20127,0.0,1.0,0.36967,0.36967,HC,ROPA LAVADO,Liquido,...,3000.0,Regular,12,4,1,3,15.63015,7.85789,5.05363,0.73934


In [48]:

###Eliminamos columnas no utilizadas
df.drop(columns=['sku_size', 'descripcion', 'cust_request_qty', 'cust_request_tn','cat1', 'cat2', 'cat3', 'brand', 'plan_precios_cuidados'], inplace = True)
df

,Timestamp,customer_id,product_id,tn,mes,quarter,fin_quarter,edad_producto,ventas_cat1,ventas_cat2,ventas_cat3,ventas_familia_producto
0,2017-01-01,10234,20524,0.05300,1,1,0,0,14.31686,4.96628,3.03194,0.25684
1,2017-02-01,10234,20524,0.00000,2,1,0,1,2.14290,0.10339,0.00000,0.00000
2,2017-03-01,10234,20524,0.01514,3,1,1,2,8.59237,2.23835,1.52777,0.04699
3,2017-04-01,10234,20524,0.00000,4,2,0,3,9.18260,4.47157,2.35257,0.00000
4,2017-05-01,10234,20524,0.00000,5,2,0,4,7.79714,0.50130,0.09348,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...
6910867,2019-12-01,10083,20127,0.24645,12,4,1,3,35.48098,20.96524,6.93741,2.10633
6910868,2019-12-01,10074,20127,20.70148,12,4,1,3,113.11048,64.95754,49.02482,22.42660
6910869,2019-12-01,10172,20127,2.58769,12,4,1,3,14.09831,11.78360,11.78360,4.31281
6910870,2019-12-01,10175,20127,0.36967,12,4,1,3,15.63015,7.85789,5.05363,0.73934


In [49]:
ventas_productos = df.groupby(['Timestamp', 'product_id', 'customer_id'])['tn'].sum().reset_index()
ventas_productos_dic_2019 = ventas_productos[ventas_productos['Timestamp'] == '2019-12-01']
ventas_productos_dic_2019 = ventas_productos_dic_2019.sort_values(by='tn', ascending = False)

lista_productos_clientes_evaluar = []
for _,fila in ventas_productos_dic_2019[:100].iterrows():
    lista_productos_clientes_evaluar.append((fila['product_id'], fila['customer_id']))
    
lista_productos_clientes_evaluar

[(20013, 10008),
 (20085, 10017),
 (20006, 10022),
 (20120, 10002),
 (20073, 10001),
 (20031, 10002),
 (20025, 10002),
 (20024, 10005),
 (20016, 10021),
 (20009, 10010),
 (20135, 10021),
 (20039, 10016),
 (20013, 10003),
 (20001, 10009),
 (20003, 10041),
 (20067, 10002),
 (20014, 10004),
 (20003, 10005),
 (20016, 10006),
 (20005, 10010),
 (20026, 10016),
 (20112, 10003),
 (20071, 10005),
 (20091, 10002),
 (20011, 10081),
 (20001, 10006),
 (20127, 10074),
 (20005, 10161),
 (20032, 10018),
 (20009, 10013),
 (20001, 10010),
 (20015, 10023),
 (20002, 10005),
 (20022, 10002),
 (20021, 10038),
 (20145, 10029),
 (20032, 10027),
 (20035, 10016),
 (20155, 10029),
 (20001, 10005),
 (20041, 10002),
 (20001, 10032),
 (20051, 10002),
 (20002, 10013),
 (20022, 10009),
 (20010, 10006),
 (20038, 10001),
 (20050, 10002),
 (20014, 10013),
 (20002, 10027),
 (20003, 10050),
 (20046, 10003),
 (20005, 10025),
 (20002, 10022),
 (20028, 10001),
 (20075, 10002),
 (20063, 10004),
 (20054, 10002),
 (20026, 10010

In [50]:
# Crear lags y rellenar NA
def crear_lags(df, lags):
    for lag in range(ventana_input, ventana_input+ lags + 1):
        df[f'tn_lag_{lag}'] = df['tn'].shift(lag)
        df[f'ventas_cat1_lag_{lag}'] = df['ventas_cat1'].shift(lag)
        df[f'ventas_cat2_lag_{lag}'] = df['ventas_cat2'].shift(lag)
        df[f'ventas_cat3_lag_{lag}'] = df['ventas_cat3'].shift(lag)
        df[f'ventas_familia_producto_lag_{lag}'] = df['ventas_familia_producto'].shift(lag)
    df.dropna(inplace = True)
    return df

def crear_dataset_supervisado(array, input_length, output_length):
    # Inicialización
    X, Y = [], []    # Listados que contendrán los datos de entrada y salida del modelo
    shape = array.shape
    if len(shape)==1: # Si tenemos sólo una serie (univariado)
        fils, cols = array.shape[0], 1
        array = array.reshape(fils,cols)
    else: # Multivariado
        fils, cols = array.shape
    # Generar los arreglos (utilizando ventanas deslizantes de longitud input_length)
    for i in range(fils-input_length-output_length + 1):
        X.append(array[i:i+input_length,0:cols])
        Y.append(array[i+input_length:i+input_length+output_length,-1].reshape(output_length,1))

    # Convertir listas a arreglos de NumPy
    X = np.array(X)
    Y = np.array(Y)

    return X, Y


def error_pred_sobre_venta(y_true, y_pred):
    # Seleccionar solo el segundo valor de la ventana de salida
    y_true_second = y_true[:, 1]
    y_pred_second = y_pred[:, 1]
    
    abs_diff = K.abs(y_true_second - y_pred_second)
    sum_abs_diff = K.sum(abs_diff)
    sum_y_true = K.sum(y_true_second)
    return sum_abs_diff / sum_y_true

# Definir función para crear y entrenar el modelo
def crear_modelo(ventana_input, ventana_output, lstm_units=[128,64, 32], dropout_rate=0.25):
    model = Sequential()
    model.add(Input(shape=(ventana_input, X.shape[2])))
    for units in lstm_units[0:-1]:
        model.add(LSTM(units, return_sequences = True))
        model.add(Dropout(dropout_rate))
    model.add(LSTM(units, return_sequences = False))
    model.add(Dropout(dropout_rate))
    model.add(Dense(ventana_output))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=[error_pred_sobre_venta])
    return model


In [59]:
EPOCHS = 100
BATCH_SIZE = 32

#Cantidad de productos pendientes de predecir
cant_productos_pendientes = len(df['product_id'].unique())

# Entrenar y predecir
i = 0
for producto, cliente in lista_productos_clientes_evaluar:
    i += 1
    ventas_mes_por_producto_cliente = df[(df['product_id'] == producto) & (df['customer_id'] == cliente)].copy()
    timestamp_primera_compra = min(ventas_mes_por_producto_cliente['Timestamp'])
    if timestamp_primera_compra <= '2018-12-01':
        ventas_mes_por_producto_cliente.drop(columns=['product_id', 'customer_id'], inplace=True)
        ventas_mes_por_producto_cliente.set_index('Timestamp', inplace = True)

        dict_scalers = {}
        for columna in ['tn', 'edad_producto', 'ventas_cat1', 'ventas_cat2', 'ventas_cat3', 'ventas_familia_producto']:
            scaler = RobustScaler()
            ventas_mes_por_producto_cliente[columna] = scaler.fit_transform(ventas_mes_por_producto_cliente[columna].values.reshape(-1,1))
            dict_scalers[columna] = scaler

        ventas_mes_por_producto_cliente = crear_lags(ventas_mes_por_producto_cliente, lags)
        X, Y = crear_dataset_supervisado(np.array(ventas_mes_por_producto_cliente), ventana_input, ventana_output)
        
        early_stop = EarlyStopping(monitor='loss', patience=20)
        reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=10, min_lr=1e-6)
        
        model = crear_modelo(ventana_input, ventana_output)

        history = model.fit(X, Y, epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=[early_stop, reduce_lr], verbose=0)
        
        error_modelo = history.history['error_pred_sobre_venta']
        
        print(f'Error Producto {producto} - Cliente {cliente}: {error_modelo[-1]}')
        y_pred_s = model.predict(X[-1].reshape(1,X.shape[1],X.shape[2]), verbose=0)
        y_pred = dict_scalers['tn'].inverse_transform(y_pred_s)
        prediccion_mes_2 = max(0,y_pred.flatten()[1])
        
    else:
        prediccion_mes_2 = ventas_mes_por_producto_cliente['tn'].mean()
    nueva_prediccion = pd.DataFrame({'product_id': [producto], 'customer_id': [cliente], 'prediccion': [prediccion_mes_2]})
    productos_con_prediccion = pd.concat([productos_con_prediccion, nueva_prediccion], axis = 0)
    productos_con_prediccion.to_csv(os.path.join(carpeta_exp,nombre_archivo_log), index = False)

Error Producto 20013 - Cliente 10008: 0.6417288184165955
Error Producto 20085 - Cliente 10017: 0.544558048248291
Error Producto 20006 - Cliente 10022: 0.5191431045532227
Error Producto 20120 - Cliente 10002: 0.7864448428153992
Error Producto 20073 - Cliente 10001: 1.997854471206665
Error Producto 20031 - Cliente 10002: 19.469951629638672


KeyboardInterrupt: 

In [82]:
productos_con_prediccion = pd.read_csv(os.path.join(carpeta_exp,nombre_archivo_log))

dataset_completo = pd.read_csv(os.path.join(carpeta_datasets, 'df_producto_cliente_completo.csv'))

# Preparar ventas por producto, cliente, mes
ventas_producto_cliente_mes = dataset_completo.groupby(['Timestamp', 'product_id', 'customer_id'])['tn'].sum().reset_index()
ventas_producto_cliente_mes

ventas_2019 = ventas_producto_cliente_mes[ventas_producto_cliente_mes['Timestamp'] >= '2019-01-01']
ventas_2019

promedio_ventas_2019 = ventas_2019.groupby(['product_id', 'customer_id'])['tn'].mean().reset_index()
promedio_ventas_2019

,product_id,customer_id,tn
0,20001,10001,214.804002
1,20001,10002,47.015213
2,20001,10003,121.867646
3,20001,10004,170.327932
4,20001,10005,10.507569
...,...,...,...
262800,21276,10428,0.000836
262801,21276,10456,0.001039
262802,21276,10462,0.000298
262803,21276,10495,0.000148


In [90]:
for _, fila in productos_con_prediccion.iterrows():
    producto = int(fila['product_id'])
    cliente = int(fila['customer_id'])
    prediccion = fila['prediccion']
    promedio_ventas_2019.loc[(promedio_ventas_2019['product_id'] == producto) & (promedio_ventas_2019['customer_id'] == cliente),'tn'] = prediccion
    
promedio_ventas_2019_producto = promedio_ventas_2019.groupby(['product_id'])['tn'].sum().reset_index()
promedio_ventas_2019_producto

promedio_ventas_2019_producto.to_csv(os.path.join(carpeta_exp, 'LSTM_producto_cliente.csv'), index = False)